In [6]:
import numpy as np
import pandas as pd
import glob
from organoid_tracking import organoid_tracking
import os
from itertools import repeat
from natsort import natsorted
from pathlib import Path


## analyzing an image sequence

we get the organoid properties sequentially and then link the individual organoids together using trackpy.

In [7]:
start_dir = '/Users/clemence/Documents_Clémence/Analysis/Tracking algorithm/Tracking-seg_Aaron-Gus_CNV/To run'

pattern   = "*/masks_predicted/*.tif"
fnames = []
validation_directories = []

for dir,_,_ in os.walk(start_dir):
    fnames.extend(glob.glob(os.path.join(dir,pattern)))

for fname in fnames:

    experiment_directory = '/'.join(fname.split('/')[:-2])

    validation_directory = experiment_directory + '/' + 'validation_image'
    validation_directories.append(validation_directory)

    if not os.path.exists(validation_directory):
        os.makedirs(validation_directory)

savedir = os.path.join(start_dir, 'organoid_statistics')

if not os.path.exists(savedir):
    os.makedirs(savedir)

In [3]:
os.path.basename(fname)

'CNV010_20210606_175.tif'

In [8]:

movie_frames = map(organoid_tracking.movie_analysis, natsorted(fnames), validation_directories)

file_data = pd.DataFrame()
file_data = pd.concat(list(movie_frames), 
                      ignore_index=True)
file_data['unique_organoid'] = file_data['movie_name']+ ':' + file_data['particle'].astype(str) 

Frame 48: 2 trajectories present.


In [9]:
pxtomu = 0.9075
frametohr = 0.5/24

file_data['velocity'] *= pxtomu/frametohr
file_data['area'] *= pxtomu**2
file_data['perimeter'] *= pxtomu
file_data['cumulative_displacement'] *= pxtomu
file_data['absolute_displacement_x'] *= pxtomu
file_data['circularity'] = 4*np.pi*(file_data['area']/file_data['perimeter']**2)

In [10]:
def get_organoid_statistics(file_data):

    organoid_statistics = pd.DataFrame(columns = ['movie_name', 'particle'])

    for single_particle in file_data.unique_organoid.unique():

        unique_organoid_frame = file_data[file_data.unique_organoid == single_particle].sort_values('frame')
        
        assert len(unique_organoid_frame.movie_name.unique()) == 1
        assert len(unique_organoid_frame.particle.unique()) == 1

        single_organoid_statistics = pd.DataFrame(columns = ['movie_name',
                                                            'particle',
                                                            'unique_organoid',
                                                            'velocity_std',
                                                            'velocity_mean',
                                                            'perimeter',
                                                            'area',
                                                            'circularity'])

        single_organoid_statistics.loc[0, 'movie_name'] = unique_organoid_frame.movie_name.unique()[0]
        single_organoid_statistics.loc[0, 'particle'] = unique_organoid_frame.particle.unique()[0]
        single_organoid_statistics.loc[0, 'unique_organoid'] = single_particle
        single_organoid_statistics.loc[0, 'N'] = len(unique_organoid_frame)
        single_organoid_statistics.loc[0, 'cumulative_displacement'] = unique_organoid_frame['cumulative_displacement'].max()
        single_organoid_statistics.loc[0, 'absolute_displacement_x'] = unique_organoid_frame.loc[unique_organoid_frame.index.max(), 'absolute_displacement_x']
        single_organoid_statistics.loc[0, 'velocity_std'] = unique_organoid_frame['velocity'].std()
        single_organoid_statistics.loc[0, 'velocity_mean'] = unique_organoid_frame['velocity'].mean()
        single_organoid_statistics.loc[0, 'perimeter'] = unique_organoid_frame['perimeter'].mean()
        single_organoid_statistics.loc[0, 'area'] = unique_organoid_frame['area'].mean()
        single_organoid_statistics.loc[0, 'circularity'] = 4*np.pi*(unique_organoid_frame['area']/unique_organoid_frame['perimeter']**2).mean()

        organoid_statistics = pd.concat([organoid_statistics, single_organoid_statistics])

    return organoid_statistics

In [12]:
for ename in file_data.experience_name.unique():
    for cname in file_data.condition_name.unique():

        print(cname, ename)
        
        df = file_data[(file_data.experience_name == ename) & (file_data.condition_name == cname)]

        Path("".join([savedir,'/trajectory_properties/',ename,'/',cname])).mkdir(parents=True, exist_ok=True)
        df.to_csv("".join([savedir,'/trajectory_properties/',ename,'/',cname,'/','trajectory_properties.csv']), index=False)

        of = get_organoid_statistics(df)
        Path("".join([savedir,'/average_organoid_properties/',ename,'/',cname])).mkdir(parents=True, exist_ok=True)
        of.to_csv("".join([savedir,'/average_organoid_properties/',ename,'/',cname,'/','average_organoid_properties.csv']), index=False)

        absolute_displacement = df.pivot(index = 'frame', columns = 'unique_organoid', values = 'absolute_displacement_x').fillna(method='ffill').dropna(axis=1, how='all').fillna(0)
        Path("".join([savedir,'/absolute_displacement/',ename,'/',cname])).mkdir(parents=True, exist_ok=True)
        absolute_displacement.to_csv("".join([savedir,'/absolute_displacement/',ename,'/',cname,'/','absolute_displacement.csv']), index=False)

        cumulative_displacement = df.pivot(index = 'frame', columns = 'unique_organoid', values = 'cumulative_displacement').fillna(method='ffill').dropna(axis=1, how='all').fillna(0)
        Path("".join([savedir,'/cumulative_displacement/',ename,'/',cname])).mkdir(parents=True, exist_ok=True)
        cumulative_displacement.to_csv("".join([savedir,'/cumulative_displacement/',ename,'/',cname,'/','cumulative_displacement.csv']), index=False)

231023-CNV-testColl 231023-CNV_test
231023-CNV-testPEG 231023-CNV_test


In [8]:
file_data.experience_name

0      231023-CNV_test
1      231023-CNV_test
2      231023-CNV_test
3      231023-CNV_test
4      231023-CNV_test
            ...       
289    231023-CNV_test
290    231023-CNV_test
291    231023-CNV_test
292    231023-CNV_test
293    231023-CNV_test
Name: experience_name, Length: 294, dtype: object